In [ ]:
%matplotlib notebook

import numpy as np

from src.PyPO.System import System

s = System()

In [ ]:
plane_focus = {
            "name"      : "plane_focus",
            "gmode"     : "uv",
            "lims_u"    : np.array([0, 0.1]),
            "lims_v"    : np.array([0, 360]),
            "gridsize"  : np.array([101, 101])
            }

s.addPlane(plane_focus)

GPODict = {                                                                                                                                                                     
    "name"      : "focus",                                                                                                                                  
    "lam"       : 0.01,                                                                                                      
    "w0x"       : 0.05,                                                                                             
    "w0y"       : 0.05,                                                                                             
    "n"         : 1,                                                                                                                             
    "E0"        : 1,                                                                                                                                  
    "dxyz"      : 0,                                                                                                 
    "pol"       : np.array([1, 0, 0])                                                                                                          
}

s.createGaussian(GPODict, "plane_focus")
s.plotBeam2D("focus", "Ex", vmin=-30, vmax=0)

s.translateGrids("plane_focus", np.array([0, 0, 100]))


In [ ]:
ellipse = {
            "name"      : "ellipsoid",
            "pmode"     : "focus",
            "gmode"     : "uv",
            "flip"      : True,
            "focus_1"   : np.array([0, 0, 100]),
            "focus_2"   : np.array([0, 0, -100]),
            "orient"    : "z",
            "ecc"       : 0.5,
            "lims_u"    : np.array([0, 10]),
            "lims_v"    : np.array([0, 360]),
            "gridsize"  : np.array([801, 801])
            }

s.addEllipse(ellipse)

plane_t = {
            "name"      : "plane_t",
            "gmode"     : "uv",
            "lims_u"    : np.array([0, 5]),
            "lims_v"    : np.array([0, 360]),
            "gridsize"  : np.array([301, 301])
            }

s.addPlane(plane_t)
s.rotateGrids("plane_t", np.array([45, 0, 0]))

parabola = {
            "name"      : "paraboloid",
            "pmode"     : "focus",
            "gmode"     : "uv",
            "focus_1"   : np.array([0, -100, 0]),
            "vertex"    : np.array([-10, -100, 0]),
            "lims_u"    : np.array([0, 0.5]),
            "lims_v"    : np.array([0, 360]),
            "gcenter"   : np.array([0,-20]),
            "gridsize"  : np.array([801, 801])
            }

s.addParabola(parabola)

plane_out = {
            "name"      : "plane_out",
            "gmode"     : "uv",
            "lims_u"    : np.array([0, 0.5]),
            "lims_v"    : np.array([0, 360]),
            "gridsize"  : np.array([301, 301])
            }

s.addPlane(plane_out)
s.rotateGrids("plane_out", np.array([0, -90, 0]))
s.translateGrids("plane_out", np.array([50, -120, 0]))

In [ ]:
focus_to_ell_PO = {
    "t_name"      : "ellipsoid",
    "s_current"   : "focus",
    "mode"        : "JMEH",
    "name_JM"     : "JM_ell",
    "name_EH"     : "EH_ell",
    "nThreads"    : 256,
    "device"      : "GPU",
    "epsilon"     : 10
}

s.runPO(focus_to_ell_PO)
#s.plotBeam2D("EH_ell", "Ex", project="xy", vmin=-30, vmax=0)

ell_to_plane_t_PO = {
    "t_name"      : "plane_t",
    "s_current"   : "JM_ell",
    "mode"        : "JMEH",
    "name_JM"     : "JM_pt",
    "name_EH"     : "EH_pt",
    "nThreads"    : 256,
    "device"      : "GPU",
    "epsilon"     : 10
}

s.runPO(ell_to_plane_t_PO)
#s.plotBeam2D("EH_pt", "Ex", project="xy", vmin=-30, vmax=0)

plane_t_to_par_PO = {
    "t_name"      : "paraboloid",
    "s_current"   : "JM_pt",
    "mode"        : "JMEH",
    "name_JM"     : "JM_par",
    "name_EH"     : "EH_par",
    "nThreads"    : 256,
    "device"      : "GPU",
    "epsilon"     : 10
}

s.runPO(plane_t_to_par_PO)
#s.plotBeam2D("EH_par", "Ex", project="xz", vmin=-30, vmax=0)

par_to_plane_out_PO = {
    "t_name"      : "plane_out",
    "s_current"   : "JM_par",
    "mode"        : "EH",
    "name_EH"     : "EH_out",
    "nThreads"    : 256,
    "device"      : "GPU",
    "epsilon"     : 10
}

s.runPO(par_to_plane_out_PO)

print(s.calcTaper("EH_out", "Ey"))

s.plotBeam2D("EH_out", "Ey", project="yz", vmin=-30, vmax=0)




In [ ]:
plane_ff = {
            "name"      : "plane_ff",
            "gmode"     : "AoE",
            "lims_Az"    : np.array([-0.7, 0.7]) * 6,
            "lims_El"    : np.array([-0.7, 0.7]) * 6 - 90,
            "gridsize"  : np.array([301, 301])
            }
s.addPlane(plane_ff)

par_to_plane_ff_PO = {
    "t_name"      : "plane_ff",
    "s_current"   : "JM_par",
    "mode"        : "FF",
    "name_EH"     : "EH_FF",
    "nThreads"    : 256,
    "device"      : "GPU",
    "epsilon"     : 10
}



s.runPO(par_to_plane_ff_PO)

s.plotBeam2D("EH_FF", "Ex", vmin=-30, vmax=0)
s.plotBeamCut("EH_FF", "Ex", vmin=-30, vmax=0)
hpbw_e, hpbw_h = s.calcHPBW("EH_FF", "Ex")
print(s.calcMainBeam("EH_FF", "Ex"))
print(hpbw_e/3600)
print(hpbw_h/3600)